<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">TRANSFORMACIÓN DEL CONJUNTO DE DATOS PARA EL DESARROLLO DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [16]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing/data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd # Manipulación y el analísis de datos.
import sqlite3 as sql # Interactuar con base de datos SQL usando Python.
from sklearn.preprocessing import MultiLabelBinarizer #Transformación entre iterables de iterables y un formato multietiqueta.
import shutil # Ayuda a automatizar el proceso de copia y eliminación de archivos y directorios.

#### CONEXIÓN CON LA BASE DE DATOS

In [21]:
#Copia de la base de datos original
db_movies = '/content/drive/MyDrive/marketing/Marketing/data/db_movies'
db_movies_copy = '/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db'
shutil.copy(db_movies, db_movies_copy)

'/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db'

In [65]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect(db_movies_copy)

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [66]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[('ratings',), ('movies',)]

#### EXPLORACIÓN DE LAS TABLAS

##### TABLA MOVIES

In [79]:
# Ejecuta la consulta SQL "SELECT * FROM movies" sobre la base de datos y carga los resultados en un DataFrame de pandas.
table_movies = pd. read_sql_query("SELECT * FROM movies", conexion)
table_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy





```
# Transformaciones requeridas en la tabla movies
  - Separación del nombre de la película y el año de estreno.
  - Separación de la columna de género en columnas binarias.
  Columna genres contiene múltiples géneros por cada fila.
```





In [80]:
#Separación del nombre de la película y el año de estreno.
separacion = """SELECT movieId, genres, TRIM(SUBSTR(title, 1, LENGTH(title) - 7)) AS titulo, SUBSTR(title, -5, 4) AS año FROM movies WHERE title LIKE '%(____)';"""
table_movies = pd.read_sql_query(separacion, conexion)
table_movies

,movieId,genres,titulo,año
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995
...,...,...,...,...
9713,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017
9714,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017
9715,193585,Drama,Flint,2017
9716,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018


In [81]:
# Separar los géneros en listas
table_movies['genres'] = table_movies['genres'].str.split('|')

# Binarizar los géneros
mlb = MultiLabelBinarizer()
generos_bin = mlb.fit_transform(table_movies['genres'])

# Crear DataFrame con los géneros binarizados
df_generos = pd.DataFrame(generos_bin, columns=mlb.classes_, index=table_movies.index)


# Unir los géneros binarizados al DataFrame original
table_movies = pd.concat([table_movies, df_generos], axis=1)

#Eliminar la columna genres
table_movies.drop('genres', axis=1, inplace=True)

# Mostrar las primeras filas del resultado
display(table_movies.head())

,movieId,titulo,año,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
# Guardar el DataFrame con las transformaciones en la base de datos de la tabla movies.
table_movies.to_sql('movies', conexion, if_exists='replace', index=False)

#Ayuda a guardar los cambios
conexion.commit()

##### TABLA RATINGS

In [85]:
# Ejecuta la consulta SQL "SELECT * FROM ratings" sobre la base de datos y carga los resultados en un DataFrame de pandas.
table_ratings = pd.read_sql_query("SELECT * FROM ratings", conexion)
table_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931




```
# Transformaciones requeridas para la tabla rating
 - La columna timestamp almacena información de fecha y hora,
 incluyendo el año, mes, día, hora, minuto y segundo, se puede separar
 para hacer consultas.
```



In [86]:
#Transformación de la columna timestampo por fecha y hora.
table_ratings['timestamp'] = pd.to_datetime(table_ratings['timestamp'], unit='s')
table_ratings['day'] = table_ratings['timestamp'].dt.day
table_ratings['month'] = table_ratings['timestamp'].dt.month
table_ratings['hour'] = table_ratings['timestamp'].dt.hour
table_ratings['year'] = table_ratings['timestamp'].dt.year

if 'date' in table_ratings.columns:
  table_ratings = table_ratings.drop('date', axis=1)

#Eliminar la columna timestampo
table_ratings.drop(columns=['timestamp'], inplace=True)

# Mostrar el DataFrame actualizado
display(table_ratings.head())


,userId,movieId,rating,day,month,hour,year
0,1,1,4.0,30,7,18,2000
1,1,3,4.0,30,7,18,2000
2,1,6,4.0,30,7,18,2000
3,1,47,5.0,30,7,19,2000
4,1,50,5.0,30,7,18,2000


In [87]:
# Guardar el DataFrame con las transformaciones en la base de datos de la tabla ratings.
table_ratings.to_sql('ratings', conexion, if_exists='replace', index=False)

#Ayuda a guardar los cambios
conexion.commit()

In [89]:
#Cierre de la conexión con la base de datos.
conexion.close()